In [1]:
import urllib
import yaml
import pandas as pd
import random
import numpy as np
from opcode import haslocal
from pprint import pprint


In [2]:
url = 'https://raw.githubusercontent.com/EktaDhobley/Algorithms-in-the-wild/main/data/student_data/final_student_merged_datasetA.csv'
student_df = pd.read_csv(url, index_col= False)

In [3]:
url = 'https://raw.githubusercontent.com/EktaDhobley/Algorithms-in-the-wild/main/data/rooms_data/final_data_bug_fixed.csv'
rooms_df = pd.read_csv(url, index_col= False)

In [4]:
#sorting the student dataset according to RUID in order to give them random priority
student_df.set_index('RUID', inplace = True)

In [5]:
allocating = dict()
available_rooms = rooms_df['hall_id'].value_counts().to_dict()
#adding new column (if the room is available or not)
rooms_df.loc[0:5330,['is_available']] = [1]

In [6]:
preference_list = dict()
for RUID, row in student_df.iterrows():
    preference_list[RUID] = row['preferred_hall_ids'].replace(' ', ",").replace('[','').replace(']','').replace('\'','').split(',') #string split

In [7]:
result = dict()

for ruid, preferences in preference_list.items():
    is_pref_found = False
    room_details = []
    available_rooms_df = pd.DataFrame()

    for pref in preferences:
        available_rooms_df = rooms_df.loc[(rooms_df['hall_id'] == pref) & rooms_df['is_available'] == 1]
        if not available_rooms_df.empty:
            #try allocating first room in the df
            is_pref_found = True
            break
    
    if not is_pref_found:
        #allocate random available room
        available_rooms_df = rooms_df.loc[rooms_df['is_available'] == 1]
    
    if not available_rooms_df.empty:
        #try allocating first room in the df
        allocated_room = available_rooms_df.iloc[0]
            
        room_details.append(allocated_room['hall_id'])
        room_details.append(allocated_room['building_id'])
        room_details.append(allocated_room['room_id'])
            
        rooms_df.loc[allocated_room.name,'is_available'] = 0
        result[ruid] = room_details

In [9]:
df = pd.DataFrame(list(result.items()), columns=['RUID', 'Assigned Rooms'])
df.to_csv('allocated.csv')

In [11]:
#sanity check
check_set = set()
counter = 0
for ruid, rooms in result.items():
    _str = str(rooms[0]) + '_' + str(rooms[1]) + '_' + str(rooms[2])
    if _str in check_set:
        print("already contains ", _str)
    check_set.add(_str)
    counter += 1
print(counter)

print(len(check_set))

5330
5330
